<a href="https://colab.research.google.com/github/Amdev-5/IMDB-Movie-Sentiment-Analysis-using-BERT/blob/main/IMDB_Movies_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 96 kB/s 
     |████████████████████████████████| 6.8 MB 33.9 MB/s 
     |████████████████████████████████| 981 kB 45.8 MB/s 
     |████████████████████████████████| 263 kB 47.4 MB/s 
     |████████████████████████████████| 1.9 MB 35.6 MB/s 
     |████████████████████████████████| 1.2 MB 51.0 MB/s 
     |████████████████████████████████| 468 kB 45.5 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 3.3 MB 37.3 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=84da9177b4121324f9504e1728126a3d8413cc9b16ab2e48d876993cfdc687bd
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=c9fc4a4a8d7c526fb6b10018fee98787460231544562f69a34da3a06a14b7e14
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff0

In [6]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [9]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype = str)

In [10]:
data_train

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg
...,...,...
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg


In [11]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype = str)
data_test

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg
...,...,...
24995,This fanciful horror flick has Vincent Price p...,neg
24996,"The Intruder (L'Intrus), a film directed by Fr...",pos
24997,Holy crap. This was the worst film I have seen...,neg
24998,Clocking in at an interminable three hours and...,neg


In [15]:
(x_Train,y_train),(x_test,y_test),preprocess = text.texts_from_df(train_df = data_train,
                   text_column = 'Reviews',
                   label_columns = 'Sentiment',
                   val_df = data_test,
                   maxlen = 400,
                   preprocess_mode = 'bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [18]:
x_Train[0].shape

(25000, 400)

In [19]:
model = text.text_classifier(name = 'bert',
                             train_data = (x_Train,y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [20]:
#Learning rate
#with k-train it finds optimal learning rate
learner = ktrain.get_learner(model = model,
                             train_data = (x_Train,y_train),
                             val_data = (x_test,y_test),
                             batch_size = 6
                             )

In [22]:
learner.lr_find()
learner.lr_plot()



simulating training for different learning rates... this may take a few moments...


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/1024


ValueError: ignored

In [23]:
#Traning 
learner.fit_onecycle(lr = 2e-5,epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - ETA: 0s - loss: 0.2569 - accuracy: 0.8931

KeyboardInterrupt: ignored

In [24]:
predictor = ktrain.get_predictor(learner.model,preprocess)

In [31]:
data = ['this movie was average. Actor was good but direction was poor, I would appreciate script overall it was okay ']

In [32]:
predictor.predict(data)

['neg']